# Imports

In [ ]:
import cvxpy as cp
import numpy as np
import random
from scipy.optimize import linprog
from tqdm import tqdm
from time import time
import matplotlib.pyplot as plt

def printf(s):
  s = '\n\033[43m'+ '\033[1m' + str(s) + '\033[0m \n'
  print(s)

# Random Test case generator

In [ ]:
def create(m,n):

  if type(n) is int:
    n = [n]

  R = []; C = []
  for i in range(len(n)):
    R.append(np.random.randint(100,size=(m,n[i])))
    C.append(np.random.randint(100,size=(m,n[i])))
  
  p = [1/len(n)]*len(n)
  return R,C,p


# DOBSS and DOBSS modification 1

In [ ]:
# Stackelberg game solution -> without change
def formulate_sg(R,C,p,M=1e3,Δ=0,lp=0):

  m,n = R.shape

  z = cp.Variable((m,n), nonneg=True)
  q = cp.Variable(n,integer=True)
  a = cp.Variable(1)

  obj = p*cp.Maximize(sum(sum(cp.multiply(R,z))))
  con = [#z<=1,q<=1, #feasibility, but this is implied by the following constraints
         
         0<=q,
         sum(sum(z))==1, #sum of all xi's=1 -> belongs to probability simplex
         sum(z.T)<=1, #individual xi's <=1 -> belongs to probability simplex

         q<=sum(z), sum(z)<=1, #as zij=xi.qj
         sum(q)==1, #sum of all qj's=1 -> belongs to probability simplex

         0<=a-sum(z.T)@C, a-sum(z.T)@C-M*(1-q)<=0, #follower's problem -> maximizing follower revenue

       ]

  if lp==1:
    x = cp.Variable(m,integer=True)
    con+=[sum(z.T)==x]

  return obj, con, z, q


# Stackelberg game solution 
def formulate_sg2(R,C,p,M=1e3,Δ=0,lp=0):

  m,n = R.shape

  z = cp.Variable((m,n), nonneg=True)
  q = cp.Variable(n,integer=True)
  a = cp.Variable(1)

  obj = p*cp.Maximize(sum(sum(cp.multiply(R,z))))
  con = [
         
        #  z<=1,q<=1, #feasibility, but this is implied by the following constraints
         0<=q,
         sum(sum(z))==1, #sum of all xi's=1 -> belongs to probability simplex
         sum(z.T)<=1, #individual xi's <=1 -> belongs to probability simplex

         q<=sum(z), sum(z)<=1, #as zij=xi.qj
         sum(q)==1, #sum of all qj's=1 -> belongs to probability simplex

        #  x == sum(z.T), # by definition
        #  0<=a-x@C, a-x@C-1e3*(1-q)<=0, #follower's problem -> maximizing follower revenue
         0<=a-sum(z.T)@C, a-sum(z.T)@C-M*(1-q)<=0, #follower's problem -> maximizing follower revenue
         sum(sum(cp.multiply(R,z))) <= Δ + sum(z.T)@R + M*( sum(sum(cp.multiply(C,z))) - sum(z.T)@C ) # Select the payoff of leader which minimum among all available maximizing strategies of follower
       ]

  if lp==1:
    x = cp.Variable(m,integer=True)
    con+=[sum(z.T)==x]

  return obj, con, z, q


# Bayesian Stackelberg game solution 
def formulate_bsg(R,C,p,solver=1,Δ=0,lp=0):

  for l in range(len(R)):
    if solver==0:
      o,c,z,q = formulate_sg(R[l],C[l],p[l],Δ=Δ,lp=lp)
    elif solver==1:
      o,c,z,q = formulate_sg2(R[l],C[l],p[l],Δ=Δ,lp=lp)

    if l == 0:
      Obj = o
      Con = c
      Z = [z]
      Q = [q]
    else:
      Obj += o
      Con += c
      Z +=[z]
      Q +=[q]

  return Obj,Con,Z,Q 


# Solver of the game 
def optimal_strategy(R,C,p,show=False,solver=0,Δ=0,lp=0):

  obj, con, Z, Q = formulate_bsg(R,C,p,solver=solver,Δ=Δ,lp=lp)

  if len(Z)>1:
    c2 = []
    for l in range(len(Z)):
      if l==0:
        k = sum(Z[l].T)
      else:
        c2+=[k==sum(Z[l].T)] # additional constraint for all xi's to be equal    
    con+=c2

  prob = cp.Problem(obj, con)
  # The optimal objective value is returned by `prob.solve()`.
  result = prob.solve()

  for i in range(len(Z)):
    if i == 0:
      zf = np.sum(np.asarray(Z[i].value),axis=1)/len(Z)
    else:
      zf += np.sum(np.asarray(Z[i].value),axis=1)/len(Z)


  ans = 0
  ans2 = 0
  for i in range(len(Z)):
    ans += np.sum(Z[i].value*R[i])*p[i]
    ans2 += np.sum(Z[i].value*C[i])*p[i]


  if show==True:

    print("-------------------------------------------------")
    printf('Solver='+str(solver)+'  Δ='+str(Δ))
    print("Optimal value: ",prob.value,'\n\n')
    print("Leader\'s strategy",'\n',zf,'\n\n')
    print("Follower strategies - for different types")
    for i in range(len(Z)):
      print("Player type",i," : ", Q[i].value)
    print('\n')
    print("-------------------------------------------------")

  return ans, ans2, Z # strategy profile


In [ ]:
R = [np.asarray ( [[2,1],[4,1.9]] ) ]
print(R)

C = [np.asarray(  [[2,2],[3,3]] ) ]
print(C)

p = [1]

v,f,Z = optimal_strategy(R,C,p,show=False,solver=0)


[array([[2. , 1. ],
       [4. , 1.9]])]
[array([[2, 2],
       [3, 3]])]


# DOBSS modification 2,3

In [ ]:
def formulate_sg_ir2(R,C,p,K=1/2,L=10,frac=0.10,solver=0,lp=0):

  m,n = R.shape
  # M = max(np.max(abs(R)),np.max(abs(C)))+1e3
  M=1e3

  K2=1
  if K<=1:
    K2=0
    K=1

  z = cp.Variable((m,n), nonneg=True)
  w = cp.Variable((m,n), nonneg=True)
  q = cp.Variable(n,integer=True)
  r = cp.Variable(n,integer=True)
  g = cp.Variable(n,integer=True) 
  a = cp.Variable(1)

  con = []

  if solver!=0:
    L = cp.Variable(1)
    con+=[L==a*frac]

  obj = p*cp.Maximize(sum(sum(cp.multiply(R,w))))
  con +=[
         # Constraints from DOBSS
         sum(sum(z))==1,                                                                                     # Sum of all xi's=1     -> belongs to probability simplex
         sum(z.T)<=1,                                                                                        # Individual xi's <=1   -> belongs to probability simplex
         0<=q, sum(q)==1,                                                                                    # Sum of all qj's=1     -> belongs to probability simplex
         q<=sum(z), sum(z)<=1,                                                                               # As zij=xi.qj          -> definition of zij's
         0<=a-sum(z.T)@C, a-sum(z.T)@C-M*(1-q)<=0,                                                           #           a           -> Max follower payoff for given leader strategy

         # Constraints for modelling Irrational behaviour         
         g<=1, 0<=g,                                                                                         # Truth value variables -> binary
         sum(z.T) == sum(w.T),                                                                               # As wij=xi.q'j         -> definition of wij's
         r<=sum(w),
         0<=r, sum(r)==1,
         K2*(sum(sum(cp.multiply(C,w))) - sum(w.T)@C) >= (sum(sum(cp.multiply(R,w))) - sum(w.T)@R)/K - M*g,  # Comparisions          -> the loss of follower < k*(loss of leader) 
         a - sum(sum(cp.multiply(C,w))) <= L,                                                                # Stop loss condition   -> the loss of follower <= stop loss
         a - sum(w.T)@C >= L - M*(1-g),                                                                      # Stop loss criterion   -> truth value from g 
         a - sum(w.T)@C <= L + M*g                                                                           # Stop loss criterion   -> truth value from g 
       ]


  if lp==1:
    x = cp.Variable(m,integer=True)
    con+=[sum(z.T)==x]
    con+=[sum(w.T)==x]

  return obj, con, w, z, q, g, a, r



# Bayesian Stackelberg game solution 
def formulate_bsg2(R,C,p,solver=0,K=2,L=10,frac=0.1,lp=0):

  for l in range(len(R)):

    o, c, w, z, q, g, a, r = formulate_sg_ir2(R[l],C[l],p[l],K=K,L=L,solver=solver,frac=frac,lp=lp)    

    if l == 0:
      Obj = o
      Con = c
      W = [w]
      Z = [z]
      Q = [q]
      G = [g]
      A = [a]
      Rr = [r]

    else:
      Obj += o
      Con += c
      W += [w]
      Z += [z]
      Q += [q]
      G += [g]
      A += [a]
      Rr += [r]

  return Obj,Con,W,Z,Q,G,A,Rr


def optimal_strategy2(R,C,p,show=False,K=2,L=10,solver=0,frac=0,lp=0):

  Obj,Con,W,Z,Q,G,A,Rr = formulate_bsg2(R,C,p,K=K,L=L,solver=solver,frac=frac,lp=lp)

  if len(W)>1:
    c2 = []
    for l in range(len(W)):
      if l==0:
        k = sum(W[l].T)
      else:
        c2+=[k==sum(W[l].T)] # additional constraint for all xi's to be equal    
    Con+=c2

  prob = cp.Problem(Obj,Con)
  result = prob.solve()

  ans = 0
  ans2 = 0
  for i in range(len(W)):
    ans += np.sum(W[i].value*R[i])*p[i]
    ans2 += np.sum(W[i].value*C[i])*p[i]

  for i in range(len(W)):
    if i == 0:
      wf = np.sum(np.asarray(W[i].value),axis=1)
    else:
      wf += np.sum(np.asarray(W[i].value),axis=1)
  wf = wf/len(W)


  if show == True:

    if solver == 0:
      printf('Solver='+str(solver+3)+'  K-factor='+str(K) +';  Stop loss ='+str(L))
    else:
      printf('Solver='+str(solver+3)+'  K-factor='+str(K) +';  Fractional loss ='+str(frac))

    print("-------------------------------------------------")
    print("Objective value "+str(ans))

    print("-------------------------------------------------")
    print("W")
    for i in range(len(W)):
      print("player type ",i+1)
      print(W[i].value)

    print("-------------------------------------------------")
    print("Z")
    for i in range(len(Z)):
      print("player type ",i+1)
      print(Z[i].value)
    
    print("-------------------------------------------------")
    print("Q")
    for i in range(len(Q)):
      print("player type ",i+1)
      print(Q[i].value)

    print("-------------------------------------------------")
    print("G")
    for i in range(len(G)):
      print("player type ",i+1)
      print(G[i].value)
    
    print("-------------------------------------------------")
    print("A")
    for i in range(len(A)):
      print("player type ",i+1)
      print(A[i].value)

    print("-------------------------------------------------")
    print("Rr")
    for i in range(len(Rr)):
      print("player type ",i+1)
      print(Rr[i].value)

    print("-------------------------------------------------")

    print("Leader payoff: ",ans,'\n\n')
    
    print("Follower payoff - for different types")
    for i in range(len(W)):
      print("Player type",i," : ", np.sum(W[i].value*C[i]))
    print('\n')

    print("-------------------------------------------------")

    print("Leader\'s strategy",'\n',wf,'\n\n')

    print("Follower strategies - for different types")
    L2 = []
    for i in range(len(W)):
      print("Player type",i," : ", np.sum(np.asarray(W[i].value),axis=0))
      L2.append(np.sum(np.asarray(W[i].value),axis=0))
    print('\n')
    
    for i in range(len(W)):
      print("W matrix",i," : ", np.asarray(W[i].value))
    print('\n')

    print("-------------------------------------------------")


  return ans, ans2, W 
  

# Comparing solutions


# 1

Let us understand how the follower picks his/her strategy and how is it different from the original algorithm using the simple example of the leader having a single strategy.

The leader has no choice of strategies here. And the follower can choose an appropriate strategy according to his/her parameter choices (L,K)

The same concept can be extended to cases when the leader has multiple pure strategies, and also he is able to select mixed strategies.



In [ ]:
R = [np.asarray ( [[5,4.5,3,0,6]] ) ]
print(R)

C = [np.asarray(  [[5,5,4.5,4,4.5]] ) ]
print(C)

p = [1]

v,f,Z = optimal_strategy(R,C,p,show=True,solver=0)
print("XXX---------------XXX---------------XXX---------------XXX\n\n\n")

v0,f0,Z0 = optimal_strategy(R,C,p,show=True,solver=1,Δ=0)
print("XXX---------------XXX---------------XXX---------------XXX\n\n\n")

v1,f1,Z1 = optimal_strategy2(R,C,p,show=True,K=4,L=2,solver=0)
print("XXX---------------XXX---------------XXX---------------XXX\n\n\n")

v2,f2,Z2 = optimal_strategy2(R,C,p,show=True,K=4.4,L=1.1,solver=0)
print("XXX---------------XXX---------------XXX---------------XXX\n\n\n")

v3,f3,Z3 = optimal_strategy2(R,C,p,show=True,K=4.49,L=1.01,solver=0)
print("XXX---------------XXX---------------XXX---------------XXX\n\n\n")

v4,f4,Z4 = optimal_strategy2(R,C,p,show=True,K=4.6,L=0.9,solver=0)
print("XXX---------------XXX---------------XXX---------------XXX\n\n\n")

v5,f5,Z5 = optimal_strategy2(R,C,p,show=True,K=6,L=2,solver=0)
print("XXX---------------XXX---------------XXX---------------XXX\n\n\n")

v6,f6,Z6 = optimal_strategy2(R,C,p,show=True,K=4,L=0.8,solver=0)
print("XXX---------------XXX---------------XXX---------------XXX\n\n\n")

v7,f7,Z7 = optimal_strategy2(R,C,p,show=True,K=-0.5,L=0.5,solver=0)
print("XXX---------------XXX---------------XXX---------------XXX\n\n\n")


# v4,f4 = optimal_strategy2(R,C,p,show=True,K=2,frac=0.1,solver=1)
# print(v1,'\n',v2,'\n',v3,'\n',v4,'\n')


print("------------------------------------------------------------------------------------------")

print(  "{:<35}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
print("------------------------------------------------------------------------------------------")

print( "{:<35}".format("Original DOBSS") ,"{:<10} {:1} {:<10}".format(  round(v,2),'|',round(f,2)))
print( "{:<35}".format("Mod-1 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v0,2),'|',round(f0,2)))

print("------------------------------------------------------------------------------------------")


print( "{:<35}".format("Mod-2 DOBSS,K=4,L=2")    ,"{:<10} {:1} {:<10}".format(  round(v1,2),'|',round(f1,2)))
print( "{:<35}".format("Mod-2 DOBSS,K=4.4,L=1.1")    ,"{:<10} {:1} {:<10}".format(  round(v2,2),'|',round(f2,2)))
print( "{:<35}".format("Mod-2 DOBSS,K=4.49,L=1.01")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
print( "{:<35}".format("Mod-2 DOBSS,K=4.6,L=0.9")    ,"{:<10} {:1} {:<10}".format(  round(v4,2),'|',round(f4,2)))
print( "{:<35}".format("Mod-2 DOBSS,K=6,L=2")    ,"{:<10} {:1} {:<10}".format(  round(v5,2),'|',round(f5,2)))
print( "{:<35}".format("Mod-2 DOBSS,K=4,L=0.8")    ,"{:<10} {:1} {:<10}".format(  round(v6,2),'|',round(f6,2)))
print( "{:<35}".format("**Mod-2 DOBSS,K=-0.5,L=0.5")    ,"{:<10} {:1} {:<10}".format(  round(v7,2),'|',round(f7,2)))


[array([[5. , 4.5, 3. , 0. , 6. ]])]
[array([[5. , 5. , 4.5, 4. , 4.5]])]
-------------------------------------------------

Solver=0  Δ=0 

Optimal value:  5.0 


Leader's strategy 
 [1.] 


Follower strategies - for different types
Player type 0  :  [1. 0. 0. 0. 0.]


-------------------------------------------------
XXX---------------XXX---------------XXX---------------XXX



-------------------------------------------------

Solver=1  Δ=0 

Optimal value:  4.5 


Leader's strategy 
 [1.] 


Follower strategies - for different types
Player type 0  :  [0. 1. 0. 0. 0.]


-------------------------------------------------
XXX---------------XXX---------------XXX---------------XXX




Solver=3  K-factor=4;  Stop loss =2 

-------------------------------------------------
Objective value 0.0
-------------------------------------------------
W
player type  1
[[0. 0. 0. 1. 0.]]
-------------------------------------------------
Z
player type  1
[[1. 0. 0. 0. 0.]]
-----------------------------

# 2

To understand how the choices made can be different, let us look at the example below where we restrict the leader strategy choice to only pure strategies, and compare it with the results of individual pure strategies. No mixed strategies can be chosen so the action space remains finite and it is easier for our understanding.

From here, we can get a vague understanding regarding how strategies are evaluated based on the parameters (L,K) and how a strategy is being chosen for the follower by the algorithm.


In [ ]:
R = [np.asarray ( [[ 10,9,8,5,4,2],
                   [ 10, 9.5, 8.5, 5,4,0]
                   ] ) ]
print(R)

C = [np.asarray ( [[ 10,10,9,5,4.5,4],
                   [ 10,10,9,5,4,3]
                   ] ) ]
print(C)

p = [1]


print("---------------------------------------------\n")
print("Original DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
v1,f1,Z1 = optimal_strategy(R,C,p,show=False,solver=0,lp=1)
print( "{:<15}".format("Original DOBSS") ,"{:<10} {:1} {:<10}".format(  round(v1,2),'|',round(f1,2)))

print("---------------------------------------------\n")
v2,f2,Z2 = optimal_strategy(R,C,p,show=False,solver=1,Δ=0,lp=1)
print("Mod-1 DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
print( "{:<15}".format("Mod-1 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v2,2),'|',round(f2,2)))


print("---------------------------------------------\n")

print("Mod-2 DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=1,L=1,solver=0,lp=1)
print( "{:<15}".format("K=1,L=1")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=2.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=2.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=5.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=5.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=6.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=6.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=7.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=7.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))



[array([[10. ,  9. ,  8. ,  5. ,  4. ,  2. ],
       [10. ,  9.5,  8.5,  5. ,  4. ,  0. ]])]
[array([[10. , 10. ,  9. ,  5. ,  4.5,  4. ],
       [10. , 10. ,  9. ,  5. ,  4. ,  3. ]])]
---------------------------------------------

Original DOBSS
                Leader     | Follower  
Original DOBSS  10.0       | 10.0      
---------------------------------------------

Mod-1 DOBSS
                Leader     | Follower  
Mod-1 DOBSS     9.5        | 10.0      
---------------------------------------------

Mod-2 DOBSS
                Leader     | Follower  
K=1,L=1         9.5        | 10.0      
K=1,L=2.5       8.5        | 9.0       
K=1,L=5.5       5.0        | 5.0       
K=1,L=6.5       4.0        | 4.0       
K=1,L=7.5       2.0        | 4.0       


In [ ]:
print("Pure strategy 1 only")

R = [np.asarray ( [[ 10,9,8,5,4,2]
                   ] ) ]
print(R)

C = [np.asarray ( [[ 10,10,9,5,4.5,4]
                   ] ) ]
print(C)

p = [1]


print("---------------------------------------------\n")
print("Original DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
v1,f1,Z = optimal_strategy(R,C,p,show=False,solver=0,lp=1)
print( "{:<15}".format("Original DOBSS") ,"{:<10} {:1} {:<10}".format(  round(v1,2),'|',round(f1,2)))

print("---------------------------------------------\n")
v2,f2,Z = optimal_strategy(R,C,p,show=False,solver=1,Δ=0,lp=1)
print("Mod-1 DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
print( "{:<15}".format("Mod-1 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v2,2),'|',round(f2,2)))


print("---------------------------------------------\n")

print("Mod-2 DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=1,L=1,solver=0,lp=1)
print( "{:<15}".format("K=1,L=1")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=2.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=2.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=5.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=5.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=6.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=6.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=7.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=7.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))



Pure strategy 1 only
[array([[10,  9,  8,  5,  4,  2]])]
[array([[10. , 10. ,  9. ,  5. ,  4.5,  4. ]])]
---------------------------------------------

Original DOBSS
                Leader     | Follower  
Original DOBSS  10.0       | 10.0      
---------------------------------------------

Mod-1 DOBSS
                Leader     | Follower  
Mod-1 DOBSS     9.0        | 10.0      
---------------------------------------------

Mod-2 DOBSS
                Leader     | Follower  
K=1,L=1         9.0        | 10.0      
K=1,L=2.5       8.0        | 9.0       
K=1,L=5.5       5.0        | 5.0       
K=1,L=6.5       2.0        | 4.0       
K=1,L=7.5       2.0        | 4.0       


In [ ]:
print("Pure strategy 2 only")

R = [np.asarray ( [
                   [ 10, 9.5, 8.5, 5,4,0]
                   ] ) ]
print(R)

C = [np.asarray ( [
                   [ 10,10,9,5,4,3]
                   ] ) ]
print(C)

p = [1]


print("---------------------------------------------\n")
print("Original DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
v1,f1,Z = optimal_strategy(R,C,p,show=False,solver=0,lp=1)
print( "{:<15}".format("Original DOBSS") ,"{:<10} {:1} {:<10}".format(  round(v1,2),'|',round(f1,2)))

print("---------------------------------------------\n")
v2,f2,Z = optimal_strategy(R,C,p,show=False,solver=1,Δ=0,lp=1)
print("Mod-1 DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
print( "{:<15}".format("Mod-1 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v2,2),'|',round(f2,2)))


print("---------------------------------------------\n")

print("Mod-2 DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=1,L=1,solver=0,lp=1)
print( "{:<15}".format("K=1,L=1")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=2.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=2.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=5.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=5.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=6.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=6.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=7.5,solver=0,lp=1)
print( "{:<15}".format("K=1,L=7.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))



Pure strategy 2 only
[array([[10. ,  9.5,  8.5,  5. ,  4. ,  0. ]])]
[array([[10, 10,  9,  5,  4,  3]])]
---------------------------------------------

Original DOBSS
                Leader     | Follower  
Original DOBSS  10.0       | 10.0      
---------------------------------------------

Mod-1 DOBSS
                Leader     | Follower  
Mod-1 DOBSS     9.5        | 10.0      
---------------------------------------------

Mod-2 DOBSS
                Leader     | Follower  
K=1,L=1         9.5        | 10.0      
K=1,L=2.5       8.5        | 9.0       
K=1,L=5.5       5.0        | 5.0       
K=1,L=6.5       4.0        | 4.0       
K=1,L=7.5       0.0        | 3.0       
